# Functions as First-Class Values
In programming languages where functions are first-class citizens (or values), functions are treated just like any other data type. This means you can:
- Pass functions as arguments to other functions.
- Return functions as results from other functions.
- Assign functions to variables and store them in data structures like arrays, lists, or dictionaries.

> In C#, functions can be treated as first-class values through the use of `delegate` type.

In [ ]:
Func<int, int> triple = x => x * 3;
var range = Enumerable.Range(1, 3); // [1, 2, 3]
var triples = range.Select(triple);
Console.WriteLine($"[{string.Join(", ", triples)}]");

[3, 6, 9]


# Avoiding State Mutation
In the functional paradigm, state mutation is discouraged. Instead of modifying existing data, you create new versions of it. This leads to:
- Predictable behavior (no side effects).
- Easier testing and debugging.
- Better support for concurrency and parallelism.

> In C#, while mutation is allowed, you can avoid it by using immutable data patterns and functional constructs such as **LINQ**, `records`, and `readonly` collections.

In [ ]:
Func<int, bool> isOdd = x => x % 2 == 1;
int[] original = [7, 6, 1];
var sorted = original.OrderBy(x => x);
var filtered = original.Where(isOdd);
Console.WriteLine($"[{string.Join(", ", original)}]");
Console.WriteLine($"[{string.Join(", ", sorted)}]");
Console.WriteLine($"[{string.Join(", ", filtered)}]");

[7, 6, 1]
[1, 6, 7]
[7, 1]


# Why Avoiding State Mutation Is Important
Avoiding state mutation is a core principle of functional programming. Instead of changing existing data, functional code creates new data structures with the desired changes. This immutability brings several important benefits:
1. **Easier Testing and Debugging:** Since state doesn't change unexpectedly, test cases are more reliable. You don't need to track the entire history of variable changes to find a bug.
2. **Better Concurrency and Parallelism:** Immutable data structures are thread-safe by default. You can run computations in parallel without worrying about race conditions or synchronization.


# Representing Functions in C#
- Methods are the most common and idiomatic way to define and invoke functions in C#. However, they are tightly coupled with the object-oriented paradigm and do not, by themselves, enable functional programming.
- Delegates are type-safe function pointers that allow functions to be stored in variables, passed as arguments, and returned from other functions. Delegates enable higher-order programming and are foundational to functional features in C#.
- Dictionaries can represent functions as explicit mappings between inputs (keys) and outputs (values). They're particularly useful for representing non-computable or memoized functions, allowing for constant-time lookups instead of recomputation.

# Higher-order Functions
## HOFs as Delegators
**Example 1:** `Where<T>`
A classic HOF accepts a function (predicate) and invokes it to determine whether to include an item in the output.

In [1]:
public static IEnumerable<T> Where<T>(
    this IEnumerable<T> source,
    Func<T, bool> predicate)
{
    if (source == null) throw new ArgumentNullException(nameof(source));
    if (predicate == null) throw new ArgumentNullException(nameof(predicate));

    foreach (T item in source)
    {
        if (predicate(item))
        {
            yield return item;
        }
    }
}

In [2]:
List<int> numbers = [1, 2, 3, 4, 5, 6];
var evens = numbers.Where(n => n % 2 == 0);
Console.WriteLine($"[{string.Join(", ", evens)}]");

[2, 4, 6]


**Example 2:** `Cache<T>`
You can use a delegate to provide fallback behavior.

In [5]:
public class Cache<T> where T : class
{
    private readonly Dictionary<Guid, T> _storage = new();
    public T Get(Guid id)
    {
        _storage.TryGetValue(id, out var item);
        return item;
    }
    
    public T Get(Guid id, Func<T> onMiss)
    {
        var item = Get(id);
        if (item == null && onMiss != null)
        {
            item = onMiss();
            if (item != null)
            {
                _storage[id] = item;
            }
        }
        return item;
    }
}

In [ ]:
var cache = new Cache<string>();
Guid userId = Guid.NewGuid();
string user = cache.Get(userId, () => LoadUserFromDatabase(userId));
user = cache.Get(userId, () => LoadUserFromDatabase(userId));
// only fetch data from database once
Console.WriteLine(user);

static string LoadUserFromDatabase(Guid id)
{
    Console.WriteLine("Fetching from database...");
    return $"User-{id.ToString()}";
}

Fetching from database...
User-0590cc84-5ee7-4fdd-ab57-96a79b40935e


**Example 3:** Retry Logic

In [ ]:
using System.Threading;

public static class RetryHelper
{
    public static T Retry<T>(Func<T> action, int maxAttempts, int delayMs = 100)
    {
        if (action == null) throw new ArgumentNullException(nameof(action));

        int attempts = 0;
        while (true)
        {
            try
            {
                return action();
            }
            catch
            {
                attempts++;
                if (attempts >= maxAttempts)
                    throw;

                Thread.Sleep(delayMs);
            }
        }
    }
}

In [16]:
int result = RetryHelper.Retry(() =>
{
    Console.WriteLine("Trying...");
    if (new Random().Next(3) != 0) throw new Exception("Failed");
    Console.WriteLine("Complete!");
    return 42;
}, maxAttempts: 5);

Trying...
Trying...
Complete!


## Adapter Functions
Instead of running the function you pass in, adapter HOFs return a new function based on it. It helps you effectively to change the shape or behavior of the function interface.

In [23]:
public static class FunctionAdapters
{
    /// <summary>
    /// Swaps the two arguments of a binary function.
    /// For example, (x, y) => x / y becomes (y, x) => x / y.
    /// </summary>
    public static Func<T2, T1, R> SwapArgs<T1, T2, R>(Func<T1, T2, R> func)
    {
        return (t2, t1) => func(t1, t2);
    }
}

In [25]:
Func<int, double, double> divide = (x, y) => x / y;
Console.WriteLine(divide(10, 2.5)); 
Func<double, int, double> divideBy = FunctionAdapters.SwapArgs(divide);
Console.WriteLine(divideBy(2.5, 10));

4
4


## Function Factories
These are higher-order functions where the output is another function, often pre-configured with some static data or behavior. You use this pattern when you want to generate a custom function on the fly, based on input.
Suppose you're building a dynamic form validator.

In [26]:
Func<int, int, Func<string, bool>> StringLengthBetween = (min, max) =>
    str => str != null && str.Length >= min && str.Length <= max;

In [27]:
var isShortName = StringLengthBetween(2, 10);
Console.WriteLine(isShortName("Tom"));
Console.WriteLine(isShortName(""));    
Console.WriteLine(isShortName("This is too long"));

True
False
False


## Eliminate Duplication with Higher-Order Functions
Encapsulating setup/teardown logic to eliminate duplication, especially in resource management (database connections, files, sockets...)
When using Dapper, setup/teardown looks like this:
```csharp
using (var conn = new SqlConnection(connString))
{
    conn.Open();
}
```
This is repeated in every method that touches the database.
Let's abstract that into a HOF that takes care of:
- Creating and opening the connection.
- Disposing of it automatically.
- Running the provided logic.

### Step 1: Create a HOF 

In [35]:
#r "nuget: Dapper"
#r "nuget: Microsoft.Data.SqlClient"

Installed Packages Dapper, 2.1.66 Microsoft.Data.SqlClient, 6.0.2

In [37]:
using System.Data;
using Microsoft.Data.SqlClient;
using Dapper;

public static class ConnectionHelper
{
    public static R Connect<R>(string connectionString, Func<IDbConnection, R> func)
    {
        using (var conn = new SqlConnection(connectionString))
        {
            conn.Open();
            return func(conn);
        }
    }
}

In [38]:
using Dapper;

public class DbLogger
{
    private string _connectionString;

    public DbLogger(string connectionString)
        => _connectionString = connectionString;

    public void Log(string message)
        => ConnectionHelper.Connect(_connectionString, c =>
            c.Execute("sp_create_log", message, commandType: CommandType.StoredProcedure));

    public IEnumerable<string> GetLogs(DateTime since)
        => ConnectionHelper.Connect(_connectionString, c =>
            c.Query<string>("SELECT * FROM Logs WHERE Timestamp > @since", new { since }));
}

### Step 2: Generalizing `using`
The `using` keyword is not composable (it's a statement). We can write a generic `Using` function to turn it into a first-class, composable function.

In [39]:
public static class F
{
    public static R Using<TDisp, R>(TDisp disposable, Func<TDisp, R> f)
        where TDisp : IDisposable
    {
        using (disposable)
        {
            return f(disposable);
        }
    }
}

### Step 3: Use `Using` to reimplement `Connect`

In [41]:
using System.Data;
using Microsoft.Data.SqlClient;

public static class ConnectionHelper
{
    public static R Connect<R>(string connectionString, Func<IDbConnection, R> func)
        => F.Using(new SqlConnection(connectionString), conn =>
        {
            conn.Open();
            return func(conn);
        });
}